# 1. Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2. Data

In [ ]:
df = pd.read_csv("../input/2019-2020-turkey-football-club-matches/tur1920.csv", encoding="utf8")
df.drop("region", axis=1, inplace=True) # All indexes are Turkey so we don't need it.
df.rename(columns={"home_oelo": "home_elo_before",
                   "home_nelo": "home_elo_after",
                   "away_oelo": "away_elo_before",
                   "away_nelo": "away_elo_after"}, inplace=True) # This is better to me. :)
df = df.iloc[::-1] # I fixed the df according to the chronological order.
df.reset_index(inplace=True, drop=True)
print(df.head())

# 3. Elo changes of team Fenerbahçe and Galatasaray

In [ ]:
FB_all = df[np.logical_or(df["home"] == "Fenerbahçe", df["away"] == "Fenerbahçe")]
GS_all = df[np.logical_or(df["home"] == "Galatasaray", df["away"] == "Galatasaray")]
FB_all.reset_index(inplace=True, drop=True)
GS_all.reset_index(inplace=True, drop=True)

fig = plt.figure(figsize=[16,5], constrained_layout=True)
spec = fig.add_gridspec(1, 2)

ax1 = fig.add_subplot(spec[0, 0])
for i in FB_all.index:
    if FB_all["home"].iloc[i] == "Fenerbahçe": elo = (FB_all["home_elo_before"].iloc[i], FB_all["home_elo_after"].iloc[i])
    else: elo = (FB_all["away_elo_before"].iloc[i], FB_all["away_elo_after"].iloc[i])
    ax1.plot([i, i+1], elo, color="blue")

ax1.set_yticks([i for i in range(1700, 1901, 20)])
ax1.set_title("Fenerbahçe", fontsize=15)
ax1.set_xlabel("match", fontsize=13)
ax1.set_ylabel("elo", fontsize=13)
ax1.grid(linestyle="--", linewidth=1, alpha=0.4)

ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
    
ax2 = fig.add_subplot(spec[0, 1])
for i in GS_all.index:
    if GS_all["home"].iloc[i] == "Galatasaray": elo = (GS_all["home_elo_before"].iloc[i], GS_all["home_elo_after"].iloc[i])
    else: elo = (GS_all["away_elo_before"].iloc[i], GS_all["away_elo_after"].iloc[i])
    ax2.plot([i, i+1], elo, color="red")

ax2.set_title("Galatasaray", fontsize=15)
ax2.set_yticks([i for i in range(1700, 1901, 20)])
ax2.set_xlabel("match", fontsize=13)
ax2.grid(linestyle="--", linewidth=1, alpha=0.4)

ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

plt.show()

# 4. Elo difference between the beginning and end of season of all teams

In [ ]:
home_teams = list(df["home"].unique())
away_teams = list(df["away"].unique())
teams = home_teams + [i for i in away_teams if i not in home_teams]

fig = plt.figure(figsize=[16,20], constrained_layout=True)

ax = fig.add_subplot()

max_elo_gain, max_elo_loss = [None, 0, 0], [None, 0, 0]
for i in range(len(teams)):
    team = df[np.logical_or(df["home"] == teams[i], df["away"] == teams[i])]
    if team["home"].iloc[0] == teams[i]: elo = [team["home_elo_before"].iloc[0], None]
    else: elo = [team["away_elo_before"].iloc[0], None]
    if team["home"].iloc[-1] == teams[i]: elo[1] = team["home_elo_after"].iloc[-1]
    else: elo[1] = team["away_elo_after"].iloc[-1]
    ax.scatter(x=elo[0], y=teams[i], color="blue", alpha=0.8)
    ax.scatter(x=elo[1], y=teams[i], color="blue", alpha=0.8, marker=(">" if elo[1]-elo[0] >= 0 else "<"))
    ax.hlines(y=teams[i], xmin=elo[0], xmax=elo[1], color="blue", alpha=0.8)
    if elo[1]-elo[0] > max_elo_gain[2]: max_elo_gain = [teams[i], elo[1], elo[1]-elo[0]]
    if elo[1]-elo[0] < max_elo_loss[2]: max_elo_loss = [teams[i], elo[1], elo[1]-elo[0]]

ax.text(x=max_elo_gain[1]+10, y=max_elo_gain[0], s=f"max elo gain ({max_elo_gain[2]})", color="green", horizontalalignment="left")
ax.text(x=max_elo_loss[1]-10, y=max_elo_loss[0], s=f"max elo loss ({abs(max_elo_loss[2])})", color="red", horizontalalignment="right")

ax.hlines(y=teams, xmin=1200, xmax=2300, linewidth=1, linestyle="--", alpha=0.2)
ax.vlines(x=np.arange(1300, 2201, 100), ymin=0, ymax=len(teams)-1, linewidth=1, linestyle="--", alpha=0.2)

ax.set_xticks(np.arange(1200, 2301, 100))

ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["bottom"].set_position(('data',-1))
plt.show()

    